In [ ]:
# YOLOv8 Local Training & Evaluation (Pothole Dataset)

This notebook uses the local dataset under `week3/Exercise 2/Pothole_Segmentation_YOLOv8/Pothole_Segmentation_YOLOv8/` (already present),
and the training utilities in `scripts/yolo_potholes_train_colab.py` (now refactored for local runs).

Steps:
- Validate dataset directory (must contain `data.yaml`, `train/`, `valid/`).
- Select model variants (detection or segmentation).
- Train models, compare results, and create an inference video from the best model.


In [1]:
# Optional: install/upgrade packages if needed in this environment
# %pip install -U ultralytics pandas opencv-python requests

import sys, os
from pathlib import Path

# Ensure the 'scripts' folder is importable
repo_root = Path.cwd()
scripts_path = repo_root / 'scripts'
if scripts_path.exists() and str(scripts_path) not in sys.path:
    sys.path.insert(0, str(scripts_path))

print('Repo root:', repo_root)
print('Scripts path in sys.path:', str(scripts_path) in sys.path)

Repo root: c:\Users\ed\Documents\maestria\deep_learning_2025-2\week3\Exercise_2
Scripts path in sys.path: True


In [4]:
# Configure dataset directory (already present locally)
from pathlib import Path

# Path relative to this notebook's location
dataset_dir = Path('../Exercise_2/Pothole_Segmentation_YOLOv8/').resolve()
print('Dataset dir:', dataset_dir)
print('data.yaml exists:', (dataset_dir / 'data.yaml').exists())
print('train/ exists:', (dataset_dir / 'train').exists())
print('valid/ exists:', (dataset_dir / 'valid').exists())

Dataset dir: C:\Users\ed\Documents\maestria\deep_learning_2025-2\week3\Exercise_2\Pothole_Segmentation_YOLOv8
data.yaml exists: True
train/ exists: True
valid/ exists: True


In [5]:
# Import training utilities from the local script
from yolo_potholes_train_colab import (
    ensure_dataset_local, train_models, compare_results, create_inference_video
)

# Validate dataset structure
dataset_root = ensure_dataset_local(dataset_dir)
data_yaml = dataset_root / 'data.yaml'
print('Using data.yaml:', data_yaml)

Using data.yaml: C:\Users\ed\Documents\maestria\deep_learning_2025-2\week3\Exercise_2\Pothole_Segmentation_YOLOv8\data.yaml


In [ ]:
# Choose models
# For detection tasks (boxes):
models = ['yolov8n.pt', 'yolov8s.pt', 'yolov8m.pt']  # add 'yolov8l.pt', 'yolov8x.pt' if you have more GPU

# If your dataset and data.yaml are for segmentation, prefer the -seg weights:
# models = ['yolov8n-seg.pt', 'yolov8s-seg.pt', 'yolov8m-seg.pt']

epochs = 10
imgsz = 640
project = Path('runs_potholes')
project.mkdir(parents=True, exist_ok=True)
print('Project dir:', project.resolve())

Project dir: C:\Users\ed\Documents\maestria\deep_learning_2025-2\week3\Exercise_2\runs_potholes


: 

In [ ]:
# Train models
results_csv = train_models(data_yaml, models, epochs, imgsz, project)
results_csv


=== Training yolov8n.pt for 10 epochs, imgsz=640 ===
New https://pypi.org/project/ultralytics/8.3.202 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.201  Python-3.13.5 torch-2.8.0+cpu CPU (AMD Ryzen 5 3400G with Radeon Vega Graphics)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\ed\Documents\maestria\deep_learning_2025-2\week3\Exercise_2\Pothole_Segmentation_YOLOv8\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_rat

In [ ]:
# Compare results and select best model
import pandas as pd
comp = compare_results(results_csv)
display(comp)
comp_out = project / 'comparison.csv'
comp.to_csv(comp_out, index=False)
print('Saved comparison to:', comp_out.resolve())

# Pick best by mAP50-95 if available, else mAP50, else first
if 'metrics/mAP50-95(B)' in comp.columns:
    best_model_name = comp.sort_values('metrics/mAP50-95(B)', ascending=False).iloc[0]['model']
elif 'metrics/mAP50(B)' in comp.columns:
    best_model_name = comp.sort_values('metrics/mAP50(B)', ascending=False).iloc[0]['model']
else:
    best_model_name = comp.iloc[0]['model']
print('Best model:', best_model_name)
best_run_dir = project / best_model_name.replace('.pt','')
best_run_dir

In [ ]:
# Create and display inference video from the best trained model
from IPython.display import Video, display
video_path = create_inference_video(best_run_dir)
print('Inference video:', video_path)
display(Video(str(video_path), embed=True, width=960))